In [2]:
import plotly
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime
import plotly.express as px

In [117]:
# Load necessary data


patients = pd.read_csv('demo/PATIENTS.csv')
admissions = pd.read_csv("demo/ADMISSIONS.csv")
procedure_events = pd.read_csv("demo/PROCEDUREEVENTS_MV.csv")
procedure_icd = pd.read_csv("demo/PROCEDURES_ICD.csv")
diagnoses = pd.read_csv('demo/diagnoses_ICD.csv')
diagnoses_icd = pd.read_csv('demo/D_ICD_DIAGNOSES.csv')
icu_stays = pd.read_csv('demo/ICUSTAYS.csv')
prescriptions = pd.read_csv('demo/PRESCRIPTIONS.csv')

### Util functions to get age, duration date columns

In [7]:
# Calculate age function
def calculate_age(dob, dod):
    dob_date = datetime.strptime(dob, "%Y-%m-%d %H:%M:%S")
    dod_date = datetime.strptime(dod, "%Y-%m-%d %H:%M:%S")
    age = (dod_date - dob_date).days // 365
    return age

In [112]:
def calculate_hospital_stay(admit, discharge):
    admit_dt = datetime.strptime(admit, '%Y-%m-%d %H:%M:%S')
    discharge_dt = datetime.strptime(discharge, '%Y-%m-%d %H:%M:%S')

    duration = (discharge_dt - admit_dt)

    return duration.total_seconds()/(24*3600)

### Data processing (merging tables, value counting, unique value sorting etc) 

In [109]:
# Apply calculate_age function to create "age" column
patients["age"] = patients.apply(lambda row: calculate_age(row["dob"], row["dod"]), axis=1)

# Filter out rows with age greater than or equal to 120
filtered_patients = patients[patients["age"] < 120]


In [9]:
patient_admission = pd.merge(filtered_patients, admissions, on="subject_id")
admission_type_count = patient_admission["admission_type"].value_counts().to_frame().reset_index()

In [ ]:
admission_location_count = patient_admission["admission_location"].value_counts().to_frame().reset_index()

In [ ]:
# Load patients data
procedures = pd.read_csv('demo/PROCEDURES_ICD.csv')
procedures_icd = pd.read_csv('demo/D_ICD_PROCEDURES.csv')
procedures_explained = pd.merge(procedures, procedures_icd, on="icd9_code")
common_procedure_count = procedures_explained["short_title"].value_counts()[:10].to_frame().reset_index()

In [ ]:
diagnoses = pd.merge(diagnoses, diagnoses_icd, on="icd9_code")
admissions["duration"] = admissions.apply(lambda row: calculate_hospital_stay(row["admittime"], row["dischtime"]), axis=1)
diagnoses_time = pd.merge(admissions, diagnoses, on="subject_id")

In [ ]:
mortality_diseases = diagnoses_time[["hospital_expire_flag", "short_title"]]

In [115]:
icu_prescriptions = pd.merge(icu_stays, prescriptions, on="subject_id")
icu_prescriptions = icu_prescriptions[icu_prescriptions['drug_type'] == 'MAIN']
icu_prescriptions["duration"] = icu_prescriptions.apply(lambda row: calculate_hospital_stay(row["intime"], row["outtime"]), axis=1)

In [10]:
# Create a pie chart
fig = px.pie(
    admission_type_count,
    names="admission_type",
    values="count",
    title="Most type of patient admission are under emergency",
    color="admission_type",
    color_discrete_sequence=px.colors.sequential.Blues_r
)

# Show the plot
fig.show()


In [ ]:
# Create a treemap
fig = px.treemap(
    admission_location_count,
    path=["admission_location"],  
    values="count",               
    color="count",                
    color_continuous_scale=px.colors.sequential.Blugrn_r, 
    title="Admission Location Distribution"
)

# Show the plot
fig.show()


In [ ]:
# Create a horizontal bar chart
fig = px.bar(
    common_procedure_count,
    x="count",
    y="short_title",
    orientation="h",
    title="Most Common Procedures",
    labels={"count": "Frequency", "short_title": "Procedure Name"},
    color="count",
    color_continuous_scale=px.colors.sequential.Blues
)

# Update layout for better appearance
fig.update_layout(
    xaxis_title="Frequency",
    yaxis_title="Procedure Name",
    margin=dict(t=50, l=150),  # Adjust margins to fit long procedure names
    height=600                 # Increase height for better readability
)

# Show the plot
fig.show()

In [ ]:
agg_data = diagnoses_time.groupby('short_title').agg({'duration': 'mean'}).reset_index()
# agg_data = diagnoses_time.groupby('short_title').agg({'duration': 'mean'}).reset_index().head(20)


# Create a bubble chart
fig = px.scatter(agg_data, 
                 x='short_title', 
                 y='duration', 
                 size='duration', 
                 color='duration', 
                 hover_name='short_title',
                 title="Bubble Chart: Duration of Stay by Diagnosis",
                 labels={'short_title': 'Diagnosis', 'duration': 'Average Duration of Stay'},
                 template='plotly')

# Customize the layout to improve readability
fig.update_layout(
    xaxis_title="Diagnosis",
    yaxis_title="Average Duration of Stay",
    xaxis_tickangle=90,  # Rotate the x-axis labels
    showlegend=False 
)

# Show the plot
fig.show()

In [114]:
# Create a heatmap
fig = px.density_heatmap(
    mortality_diseases,
    x="short_title",
    y="hospital_expire_flag",
    title="Mortality Rate by Disease",
    labels={"short_title": "Disease", "mortality_rate": "Mortality Rate"},
    color_continuous_scale=px.colors.sequential.Reds
)

fig.update_layout(xaxis_tickangle=45)  # Rotate x-axis labels for readability
fig.show()

In [103]:
# Create a strip plot
fig = px.strip(
    icu_prescriptions,
    x="drug",
    y="duration",
    title="ICU Duration by Drug",
    labels={"drug": "Drug Name", "duration": "ICU Duration (days)"},
    color="drug",
    color_discrete_sequence=px.colors.qualitative.Dark2
)

# Update layout for better appearance
fig.update_layout(
    xaxis_title="Drug Name",
    yaxis_title="ICU Duration (days)",
    height=600,
)

fig.show()


### Grouping all plots in plotly dashboard

In [118]:
from dash import Dash, html, dcc
import plotly.express as px
import pandas as pd

# Initialize the Dash app
app = Dash(__name__)



# Create all the plots
# Pie Chart
fig_pie = px.pie(
    admission_type_count,
    names="admission_type",
    values="count",
    title="Most Types of Patient Admission are Under Emergency",
    color="admission_type",
    color_discrete_sequence=px.colors.sequential.Blues_r
)

# Treemap
fig_treemap = px.treemap(
    admission_location_count,
    path=["admission_location"],
    values="count",
    color="count",
    color_continuous_scale=px.colors.sequential.Blugrn_r,
    title="Admission Location Distribution"
)

# Horizontal Bar Chart
fig_bar = px.bar(
    common_procedure_count,
    x="count",
    y="short_title",
    orientation="h",
    title="Most Common Procedures",
    labels={"count": "Frequency", "short_title": "Procedure Name"},
    color="count",
    color_continuous_scale=px.colors.sequential.Blues
)
fig_bar.update_layout(margin=dict(t=50, l=150), height=600)

# Bubble Chart
agg_data = diagnoses_time.groupby('short_title').agg({'duration': 'mean'}).reset_index()
fig_bubble = px.scatter(
    agg_data,
    x='short_title',
    y='duration',
    size='duration',
    color='duration',
    hover_name='short_title',
    title="Bubble Chart: Duration of Stay by Diagnosis",
    labels={'short_title': 'Diagnosis', 'duration': 'Average Duration of Stay'},
)
fig_bubble.update_layout(xaxis_tickangle=90)

# Heatmap
fig_heatmap = px.density_heatmap(
    mortality_diseases,
    x="short_title",
    y="hospital_expire_flag",
    title="Mortality Rate by Disease",
    labels={"short_title": "Disease", "hospital_expire_flag": "Mortality Flag"},
    color_continuous_scale=px.colors.sequential.Reds
)
fig_heatmap.update_layout(xaxis_tickangle=45)

# Strip Plot
fig_strip = px.strip(
    icu_prescriptions,
    x="drug",
    y="duration",
    title="ICU Duration by Drug",
    labels={"drug": "Drug Name", "duration": "ICU Duration (days)"},
)
fig_strip.update_layout(height=600)

# Define the layout of the dashboard
app.layout = html.Div([
    
    # Title
    html.H1("Hospital Data Dashboard", style={"textAlign": "center"}),

    # Pie Chart Section
    html.Div([
        html.H2("Pie Chart: Admission Types"),
        dcc.Graph(figure=fig_pie)
    ], style={"margin-bottom": "50px"}),

    # Treemap Section
    html.Div([
        html.H2("Treemap: Admission Locations"),
        dcc.Graph(figure=fig_treemap)
    ], style={"margin-bottom": "50px"}),

    # Horizontal Bar Chart Section
    html.Div([
        html.H2("Horizontal Bar Chart: Common Procedures"),
        dcc.Graph(figure=fig_bar)
    ], style={"margin-bottom": "50px"}),

    # Bubble Chart Section
    html.Div([
        html.H2("Bubble Chart: Duration of Stay by Diagnosis"),
        dcc.Graph(figure=fig_bubble)
    ], style={"margin-bottom": "50px"}),

    # Heatmap Section
    html.Div([
        html.H2("Heatmap: Mortality Rate by Disease"),
        dcc.Graph(figure=fig_heatmap)
    ], style={"margin-bottom": "50px"}),

   # Strip Plot Section
   html.Div([
       html.H2("Strip Plot: ICU Duration by Drug"),
       dcc.Graph(figure=fig_strip)
   ])
])

# Run the app
if __name__ == "__main__":
    app.run_server(debug=True)
